In [1]:
# Установка зависимостей
!pip install sentence-transformers faiss-cpu keybert nltk scikit-learn dateutils spacy > /dev/null
!python -m spacy download ru_core_news_sm > /dev/null
# !python -m spacy download en_core_web_sm > /dev/null

In [2]:
# Импорты
import json
import re
from functools import lru_cache
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from sentence_transformers import SentenceTransformer
import faiss
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import SnowballStemmer
import logging
import time
import spacy
import os

In [4]:
# Конфигурация
LOG_LEVEL = logging.INFO
DRIVE_MOUNT_PATH = '/content/drive'
BASE_FOLDER_ON_DRIVE = 'ХАКАТОН' # Название основной папки на Диске
SUBFOLDER = 'json_exports' # Название подпапки с файлами

BASE_DATA_PATH = os.path.join(DRIVE_MOUNT_PATH, 'MyDrive', BASE_FOLDER_ON_DRIVE, SUBFOLDER)

# Формируем полные пути к файлам
SKILLS_FILE = os.path.join(BASE_DATA_PATH, 'skills.json')
JOBS_FILE = os.path.join(BASE_DATA_PATH, 'jobs.json')
SPECIALISTS_FILE = os.path.join(BASE_DATA_PATH, 'specialists.json')

In [5]:
# Модели
SENTENCE_TRANSFORMER_MODEL = 'paraphrase-multilingual-MiniLM-L12-v2'
SPACY_MODEL = 'ru_core_news_sm' # Русская модель
SPACY_NER_LABELS = {'ORG', 'PRODUCT', 'TECH', 'SOFTWARE', 'GPE', 'FAC'} # Расширенные метки NER

# Параметры
KEYBERT_TOP_N = 25
KEYBERT_DIVERSITY = 0.7
SEMANTIC_SIMILARITY_THRESHOLD = 0.58
SEMANTIC_MATCH_WEIGHT = 0.6
W_FAISS = 0.35
W_SKILL = 0.40
W_EXPERIENCE = 0.25

In [6]:
# Настройка логирования
logging.basicConfig(level=LOG_LEVEL, format='%(asctime)s - %(levelname)s - %(message)s')

# Инициализация
# Монтирование Google Drive
try:
    from google.colab import drive
    if not os.path.exists(os.path.join(DRIVE_MOUNT_PATH, 'MyDrive')):
         logging.info(f"Монтирование Google Drive в {DRIVE_MOUNT_PATH}...")
         drive.mount(DRIVE_MOUNT_PATH, force_remount=True)
         logging.info("Google Drive успешно смонтирован.")
    else:
         logging.info("Google Drive уже смонтирован.")
except ImportError:
    logging.info("Не в среде Colab, пропускаем монтирование Google Drive.")


try:
    nltk.data.find('tokenizers/punkt')
    logging.info("NLTK 'punkt' resource found.")
except LookupError:
    logging.info("NLTK 'punkt' resource not found. Downloading...")
    try:
        nltk.download('punkt', quiet=True); nltk.data.find('tokenizers/punkt')
        logging.info("NLTK 'punkt' resource downloaded.")
    except Exception as download_exc:
        logging.error(f"Failed to download NLTK 'punkt': {download_exc}"); exit()

logging.info("Инициализация моделей...")
init_start_time = time.time()
# Загрузка моделей машинного обучения. Оборачиваем в try-except для отлова ошибок
try:
    model = SentenceTransformer(SENTENCE_TRANSFORMER_MODEL)
    # Инициализация KeyBERT с использованием загруженной модели Sentence Transformer
    kw_model = KeyBERT(model=model)
    try:
        nlp = spacy.load(SPACY_MODEL)
        logging.info(f"Модель spaCy '{SPACY_MODEL}' загружена.")
    except OSError:
        logging.warning(f"Модель spaCy '{SPACY_MODEL}' не найдена. NER будет отключен. "
                      f"Скачайте: python -m spacy download {SPACY_MODEL}")
        nlp = None
except Exception as e:
     logging.error(f"Крит. ошибка инициализации SentenceTransformer/KeyBERT: {e}"); exit()
# Инициализация стеммеров NLTK для русского и английского языков.
try:
    stemmer_ru = SnowballStemmer('russian'); stemmer_en = SnowballStemmer('english')
    logging.info("Стеммеры инициализированы.")
except Exception as stemmer_err:
     logging.error(f"Ошибка инициализации стеммеров: {stemmer_err}"); exit()
logging.info(f"Инициализация завершена за {time.time() - init_start_time:.2f} сек.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# ---- Функции Нормализации и Вспомогательные ----
def normalize_text(text: str) -> str:
    if not text or not isinstance(text, str): return "" # Проверка на пустую строку или неверный тип
    return re.sub(r'\s+', ' ', text.lower()).strip()

# Кэшируем результаты normalize_skill, т.к. один и тот же навык может встречаться много раз
@lru_cache(maxsize=20000) # maxsize - примерный ожидаемый размер словаря навыков + синонимов
def normalize_skill(skill: str) -> str:
    """
    Нормализует строку навыка:
    1. Применяет normalize_text.
    2. Удаляет большинство знаков пунктуации (кроме букв, цифр, пробелов и дефиса внутри слова).
    3. Разделяет на части по пробелам, слешам, дефисам.
    4. Применяет стемминг к каждой части (русский или английский).
    5. Соединяет обработанные части через пробел.
    """
    skill_lower = normalize_text(skill);
    if not skill_lower: return ""
    # Удаляем все, кроме букв (русских/английских), цифр, пробелов и дефиса.
    skill_cleaned = re.sub(r'[^\w\sа-яА-ЯёЁ-]', '', skill_lower) # Оставляем дефис
    # Разделяем по пробелам, слешам или дефисам
    parts = re.split(r'[\s/-]+', skill_cleaned); stemmed_parts = []
    for part in parts:
        if not part: continue # Пропускаем пустые части после split
        part = part.strip('-') # Убираем дефисы по краям частей
        if not part: continue
        # Определяем язык и применяем стемминг
        if re.search('[а-яА-ЯёЁ]', part): stemmed_parts.append(stemmer_ru.stem(part))
        elif re.search('[a-zA-Z]', part): stemmed_parts.append(stemmer_en.stem(part))
        elif part.isdigit(): stemmed_parts.append(part)
    # Соединяем обработанные части
    stemmed = ' '.join(filter(None, stemmed_parts)).strip()
    # Не убираем короткие строки, т.к. "c#" или "go" могут быть релевантны
    # Возвращаем нормализованный навык, если он не пустой
    return stemmed if len(stemmed) > 0 else ""

In [8]:
# Определение словаря синонимов после функции нормализации
SKILL_SYNONYM_MAP = {
    # Применяем normalize_skill к каждому ключу и значению в исходном словаре синонимов
    normalize_skill(syn): normalize_skill(main) for syn, main in {
        'postgresql': 'sql', 'postgres': 'sql', 'mssql': 'sql', 'ms sql': 'sql',
        'oracle db': 'sql', 'oracle': 'sql', 'sqlite': 'sql', 'mysql': 'sql',
        'java script': 'javascript', 'js': 'javascript', 'тайпскрипт': 'typescript',
        'node js': 'nodejs', 'node.js': 'nodejs', 'с#': 'c#', 'джира': 'jira', 'гит': 'git',
        'рест': 'rest', 'rest api': 'rest', 'soap': 'rest', 'с++': 'c++', 'си++': 'c++',
        'питон': 'python', 'пайтон': 'python', 'го': 'golang', 'реак': 'react', 'reactjs': 'react',
        'ангуляр': 'angular', 'вью': 'vue', 'vuejs': 'vue', 'вордпресс': 'wordpress',
        'битрикс': 'bitrix', '1с-битрикс': 'bitrix', 'битрикс24': 'bitrix', 'б24': 'bitrix',
        'мвц': 'mvc', 'докер': 'docker', 'кубернетес': 'kubernetes', 'k8s': 'kubernetes',
        'линукс': 'linux', 'юникс': 'linux', }.items() if normalize_skill(syn) and normalize_skill(main) }
logging.info(f"Карта синонимов создана ({len(SKILL_SYNONYM_MAP)} записей).")

In [9]:
def calculate_experience_months(start: datetime, end: datetime) -> int:
    """
    Вычисляет разницу между двумя датами в месяцах.
    Ограничивает максимальный опыт 60 годами (720 месяцев).
    Возвращает 0, если даты некорректны или start > end.
    """
    if not start or not end or start > end: return 0 # Проверка корректности дат
    try: delta = relativedelta(end, start); return min(max(0, delta.years * 12 + delta.months), 720)
    except TypeError: return 0

In [ ]:
# Кэшируем эмбеддинги для текстов, чтобы не пересчитывать их каждый раз
@lru_cache(maxsize=30000) # Размер кэша для эмбеддингов текстов (вакансии, резюме)
def get_embedding(text: str):
    """Получает эмбеддинг для заданного текста с помощью SentenceTransformer."""
    if not text: return None # Возвращаем None для пустых текстов
    try: return model.encode(text) # Кодируем текст в векторное представление (эмбеддинг)
    except Exception as e: logging.warning(f"Ошибка кодирования '{text[:50]}...': {e}"); return None # Возвращаем None в случае ошибки

In [10]:
# ---- Функции Загрузки и Парсинга ----
def load_data(skills_file=SKILLS_FILE, jobs_file=JOBS_FILE, specialists_file=SPECIALISTS_FILE):
    """
    Загружает данные о навыках, вакансиях и специалистах из JSON-файлов.
    Преобразует и подготавливает данные для дальнейшей обработки.
    Возвращает словари и списки с обработанными данными.
    """
    logging.info("Загрузка данных...")
    load_start_time = time.time() # Засекаем время начала загрузки
    data = {} # Инициализируем словарь для хранения сырых данных из файлов

    # Цикл для загрузки каждого файла (навыки, вакансии, специалисты)
    for name, fp in [('skills', skills_file), ('jobs', jobs_file), ('specialists', specialists_file)]:
        # Проверяем, существует ли файл по указанному пути
        if not os.path.exists(fp):
            logging.error(f"Файл не найден: {fp}")
            # Возвращаем пустые структуры, если критичный файл отсутствует
            return {}, [], {}, {}, {}
        try:
            with open(fp, 'r', encoding='utf-8') as f:
                # Загружаем JSON данные из файла в словарь `data`
                data[name] = json.load(f)
            logging.info(f"Файл '{fp}' загружен ({len(data[name])} записей).")
        except Exception as e:
            logging.error(f"Ошибка чтения/JSON {fp}: {e}")
            # Возвращаем пустые структуры в случае ошибки чтения/парсинга
            return {}, [], {}, {}, {}

    # Извлекаем списки данных из словаря `data`, используем пустые списки по умолчанию
    skills, jobs, specialists = data.get('skills', []), data.get('jobs', []), data.get('specialists', [])

    # Инициализируем структуры для хранения обработанных данных о навыках:
    skill_db = {}
    norm_map = {}
    orig_map_norm = {}

    # Обрабатываем каждый навык из списка `skills`
    for s in skills:
        try:
            # Извлекаем ID навыка
            s_id = s['_id']['$oid']
            # Извлекаем и нормализуем (приводим к нижнему регистру, убираем лишние пробелы) русское и английское названия
            ru = normalize_text(s.get('name',{}).get('ru',''))
            en = normalize_text(s.get('name',{}).get('en',''))
            # Применяем нормализацию (стемминг, удаление пунктуации) к названиям
            n_ru = normalize_skill(ru) if ru else ''
            n_en = normalize_skill(en) if en else ''
            # Сохраняем оригинальные и нормализованные названия в `skill_db` по ID
            skill_db[s_id] = {'ru': ru, 'en': en, 'norm_ru': n_ru, 'norm_en': n_en}
            # Определяем первичное нормализованное имя (русское или английское)
            p_norm = n_ru or n_en
            # Если первичное нормализованное имя существует...
            if p_norm:
                # ...добавляем его в карту `norm_map` (нормализованное -> ID)
                norm_map[p_norm] = s_id
                # ...и в карту `orig_map_norm` (нормализованное -> оригинальные имена)
                orig_map_norm[p_norm] = {'ru': ru, 'en': en}
            # Определяем вторичное нормализованное имя (если первичное было русское, то английское, и наоборот)
            s_norm = n_en if p_norm == n_ru else n_ru
            # Если вторичное имя существует, отличается от первичного и еще не добавлено в `norm_map`...
            if s_norm and s_norm != p_norm and s_norm not in norm_map:
                # ...добавляем его тоже, чтобы оба языка вели к одному ID
                norm_map[s_norm] = s_id
        except Exception as e:
            # Лог: Предупреждение о пропуске навыка из-за ошибки при обработке
            logging.warning(f"Пропуск навыка {s.get('_id')}: {e}")
            continue # Переходим к следующему навыку
    # Лог: Информация о количестве обработанных навыков и созданных нормализованных форм
    logging.info(f"Навыки: {len(skill_db)}. Норм: {len(norm_map)}")

    # Инициализируем словарь для хранения обработанных данных о вакансиях
    job_data = {}

    # Обрабатываем каждую вакансию из списка `jobs`
    for job in jobs:
        try:
            # Извлекаем ID вакансии
            j_id = job['_id']['$oid']
            # Инициализируем списки для текстов требований, задач и обязательных требований
            req_t, task_t, mand_t = [], [], []
            # Собираем тексты требований (русский и английский)
            for r in job.get('requirements',[]):
                t_ru = r.get('requirement',{}).get('ru','')
                t_en = r.get('requirement',{}).get('en','')
                ft = f"{t_ru} {t_en}".strip() # Объединяем и убираем пробелы по краям
                if ft: req_t.append(ft) # Добавляем непустой текст требования
                # Если требование помечено как обязательное, добавляем его текст в `mand_t`
                if r.get('mandatory', False):
                    mand_t.append(ft)
            # Собираем тексты задач (русский и английский)
            for t in job.get('tasks',[]):
                t_ru = t.get('name',{}).get('ru','')
                t_en = t.get('name',{}).get('en','')
                task_t.append(f"{t_ru} {t_en}".strip()) # Добавляем непустой текст задачи

            # Собираем все текстовые части вакансии (название, требования, задачи, описание) в один список
            f_parts = [
                job.get('name',{}).get('ru',''), job.get('name',{}).get('en',''),
                "Требования:", *req_t,
                "Задачи:", *task_t,
                "Описание:", job.get('description',{}).get('ru',''), job.get('description',{}).get('en','')
            ]
            # Объединяем все части в один текст и нормализуем его
            f_text = normalize_text(" ".join(filter(None, f_parts)))
            # Извлекаем требования к опыту (навык: месяцы) из текста требований вакансии
            req_exp = parse_experience_requirements_from_text(' '.join(req_t), norm_map, skill_db)
            # Сохраняем обработанные данные вакансии в `job_data` по ID
            job_data[j_id] = {
                'name': normalize_text(f"{job.get('name',{}).get('ru','')} / {job.get('name',{}).get('en','')}") , # Нормализованное название
                'full_text': f_text, # Полный нормализованный текст вакансии
                'required_experience': req_exp # Словарь требований к опыту
            }
        except Exception as e:
            # Лог: Предупреждение о пропуске вакансии из-за ошибки
            logging.warning(f"Пропуск вакансии {job.get('_id')}: {e}")
            continue # Переходим к следующей вакансии
    logging.info(f"Вакансии: {len(job_data)}")

    # Инициализируем список для хранения обработанных данных о специалистах
    specialists_data = []

    # Обрабатываем каждого специалиста из списка `specialists` с использованием индекса
    for spec_idx, spec in enumerate(specialists):
        try:
            # Извлекаем ID специалиста
            s_id = spec['_id']['$oid']
            # Получаем список ID навыков специалиста
            skills_ids = spec.get('skills',[])
            # Инициализируем множества для хранения:
            cand_disp = set() # Оригинальные названия навыков (для отображения)
            cand_norm = set() # Нормализованные названия навыков (для сопоставления)
            # Обрабатываем каждый ID навыка специалиста
            for sk_id in skills_ids:
                # Если информация о навыке есть в нашей базе `skill_db`...
                if sk_id in skill_db:
                    info = skill_db[sk_id] # Получаем информацию о навыке
                    # Определяем имя для отображения (русское или английское)
                    d_name = info['ru'] or info['en']
                    if d_name: cand_disp.add(d_name) # Добавляем в множество для отображения
                    # Добавляем нормализованные названия (если есть) в множество для сопоставления
                    if info['norm_ru']: cand_norm.add(info['norm_ru'])
                    if info['norm_en']: cand_norm.add(info['norm_en'])

            # Инициализируем структуры для обработки опыта:
            exp_months = {} # Словарь: нормализованный навык -> суммарный опыт в месяцах
            tot_exp_parts = [] # Список для сбора текстовых описаний всего опыта

            # Обрабатываем каждую запись об опыте работы специалиста
            for exp in spec.get('experience',[]):
                try:
                    # Извлекаем даты начала и конца работы (в формате ISO)
                    s_str = exp.get('start',{}).get('$date')
                    # Конец может отсутствовать (текущее место работы)
                    e_str = exp.get('end',{}).get('$date') if exp.get('end') else None
                    # Преобразуем строки дат в объекты datetime (убираем Z и информацию о таймзоне)
                    start = datetime.fromisoformat(s_str.replace('Z','+00:00')).replace(tzinfo=None) if s_str else None
                    # Если дата окончания не указана, используем текущее время
                    end = datetime.fromisoformat(e_str.replace('Z','+00:00')).replace(tzinfo=None) if e_str else datetime.now().replace(tzinfo=None)

                    # Если даты корректны...
                    if start and end:
                        # ...вычисляем длительность опыта в месяцах
                        months = calculate_experience_months(start, end)
                        # Пропускаем, если опыт некорректен или равен нулю
                        if months <= 0: continue

                        # Собираем текстовое описание из должности, компании и описания работы (RU/EN)
                        exp_p = [ exp.get('position',{}).get(lang,'') for lang in ['ru','en'] ] + \
                                [ exp.get('company',{}).get(lang,'') for lang in ['ru','en'] ] + \
                                [ exp.get('description',{}).get(lang,'') for lang in ['ru','en'] ]
                        # Нормализуем собранный текст опыта
                        e_text = normalize_text(" ".join(filter(None, exp_p)))
                        # Пропускаем, если текст пуст
                        if not e_text: continue
                        # Добавляем текст этого опыта в общий список текстов опыта
                        tot_exp_parts.append(e_text)
                        # Инициализируем множество для найденных в этом тексте нормализованных навыков
                        found_norm = set()
                        # Ищем упоминания известных навыков в тексте опыта
                        for n_skill, db_s_id in norm_map.items():
                             # Пропускаем, если ID навыка нет в базе (маловероятно, но для надежности)
                             if db_s_id not in skill_db: continue
                             s_info = skill_db[db_s_id]
                             # Формируем варианты написания навыка (нормализованное, русское, английское)
                             variants = {n_skill, s_info['ru'], s_info['en']}
                             # Проверяем каждый вариант
                             for var in variants:
                                 # Если вариант не пустой и найден в тексте опыта (как отдельное слово, без учета регистра)...
                                 if var and re.search(r'\b'+re.escape(var)+r'\b', e_text, re.IGNORECASE):
                                     found_norm.add(n_skill) # ...добавляем нормализованный навык в найденные
                                     break # Переходим к следующему навыку из norm_map
                        # Суммируем месяцы опыта для каждого найденного навыка
                        for n_skill in found_norm:
                            if n_skill:
                                exp_months[n_skill] = exp_months.get(n_skill, 0) + months
                except Exception as e_exp:
                    # Лог: Отладочное сообщение об ошибке при обработке конкретного места работы
                    logging.debug(f"Ошибка опыта {s_id} в {exp.get('company',{})}: {e_exp}")
                    continue # Переходим к следующей записи опыта

            # Собираем полный текст профиля специалиста для эмбеддинга
            f_spec_parts = [
                spec.get('name',{}).get('ru',''), # Имя
                "Навыки:", "; ".join(sorted(list(cand_disp))), # Список навыков для отображения
                "Опыт:", ". ".join(tot_exp_parts), # Объединенные тексты опыта
                "О себе:", spec.get('aboutMe',{}).get('ru',''), spec.get('aboutMe',{}).get('en','')
            ]
            # Объединяем все части и нормализуем
            f_spec_text = normalize_text(" ".join(filter(None, f_spec_parts)))

            # Добавляем обработанные данные специалиста в список `specialists_data`
            specialists_data.append({
                'id': s_id, # ID специалиста
                'name': spec.get('name',{}).get('ru','Unknown'),
                'skills_list_ru': sorted(list(cand_disp)), # Отсортированный список оригинальных названий навыков
                'skills_list_norm': sorted(list(cand_norm)), # Отсортированный список нормализованных названий навыков
                'experience_per_skill_months': exp_months, # Словарь: нормализованный навык -> опыт в месяцах
                'full_text_for_embedding': f_spec_text # Полный нормализованный текст для создания эмбеддинга
            })
        except Exception as e_spec:
            logging.warning(f"Ошибка кандидата {spec.get('_id')}: {e_spec}")
            continue # Переходим к следующему специалисту

        if (spec_idx+1) % 100 == 0:
            logging.info(f"Обработано {spec_idx+1}/{len(specialists)} кандидатов...")

    logging.info(f"Кандидаты: {len(specialists_data)}")
    logging.info(f"Загрузка данных завершена за {time.time() - load_start_time:.2f} сек.")

    # Возвращаем все подготовленные структуры данных
    return job_data, specialists_data, skill_db, norm_map, orig_map_norm

In [12]:
def parse_experience_requirements_from_text(text: str, norm_map: dict, skill_db: dict) -> dict:
    """
    Извлекает требования к опыту (навык, месяцы) из текста вакансии.
    Ищет упоминания навыков и рядом с ними указания на количество лет/месяцев опыта.
    Приоритет отдается ближайшему предшествующему навыку.

    """
    if not text: return {} # Возвращаем пустой словарь, если текст пустой
    # Паттерны для поиска упоминаний опыта (годы/месяцы).
    pats = [(r'(\d+)\s*\+\s*(?:лет|год|года|years?)',12),
            (r'(?:от|более|не|минимум|from|more|at least)\s*(?P<val>\d+)\s*(?:лет|год|года|years?)',12),
            (r'(\d+)\s*-\s*\d+\s*(?:лет|год|года|years?)',12),
            (r'(?<![\d-])(\d+)\s*(?:лет|год|года|years?)(?![\w-])',12),
            (r'(\d+)\s*\+\s*(?:месяц|месяца|месяцев|months?)',1),
            (r'(?:от|более|не|минимум|from|more|at least)\s*(?P<val>\d+)\s*(?:месяц|месяца|месяцев|months?)',1),
            (r'(\d+)\s*-\s*\d+\s*(?:месяц|месяца|месяцев|months?)',1),
            (r'(?<![\d-])(\d+)\s*(?:месяц|месяца|месяцев|months?)(?![\w-])',1)]

    req_exp, proc_txt = {}, normalize_text(text)
    sk_pos = []

    # Находим все упоминания известных навыков и их позиции
    for n_sk, s_id in norm_map.items():
        if s_id not in skill_db: continue
        inf = skill_db[s_id]
        # Проверяем нормализованное имя, русское и английское
        vars_ = {n_sk, inf['ru'], inf['en']}
        for v in vars_:
            if v:
                try:
                    # Используем стандартный цикл for внутри try
                    for m in re.finditer(r'\b' + re.escape(v) + r'\b', proc_txt, re.IGNORECASE):
                        sk_pos.append((n_sk, m.start(), m.end()))
                except re.error:
                    # Ошибка регулярного выражения для этого варианта, пропускаем
                    continue

    if not sk_pos: return {} # Если не нашли ни одного навыка, выходим

    sk_pos.sort(key=lambda x: x[1]) # Сортируем найденные навыки по позиции

    # Находим все упоминания опыта и их позиции
    exp_matches = []
    for p, mult in pats:
        try:
            for m in re.finditer(p, proc_txt, re.IGNORECASE):
                 value_str = m.group('val') if 'val' in m.groupdict() else m.group(1)
                 exp_matches.append((int(value_str), mult, m.start(), m.end()))
        except (ValueError, IndexError, re.error):
            continue # Пропускаем ошибки парсинга или невалидные regex

    if not exp_matches: return {} # Если не нашли упоминаний опыта, выходим

    exp_matches.sort(key=lambda x: x[2]) # Сортируем найденный опыт по позиции

    # Сопоставляем опыт с ближайшим предшествующим навыком
    last_sk_idx = -1
    proc_spans = set() # Множество для отслеживания обработанных интервалов опыта
    for v, mult, start, end in exp_matches:
        if (start, end) in proc_spans: continue # Пропускаем уже обработанный опыт

        best_sk, min_d = None, 60 # Ищем ближайший навык в пределах 60 символов
        cur_sk_idx = last_sk_idx + 1 # Начинаем поиск навыка со следующего после предыдущего найденного

        while cur_sk_idx < len(sk_pos):
            n_sk, sk_s, sk_e = sk_pos[cur_sk_idx]
            if sk_e <= start:
                dist = start - sk_e # Расстояние от конца навыка до начала опыта
                if dist < min_d: # Если нашли более близкий навык
                    min_d, best_sk = dist, n_sk
                # Обновляем last_sk_idx, даже если этот навык не самый близкий,
                # чтобы следующий поиск опыта начался с него
                last_sk_idx = cur_sk_idx
                cur_sk_idx += 1
            else:
                # Текущий навык начинается ПОСЛЕ начала опыта, значит,
                # для данного опыта мы уже не найдем предшествующих навыков дальше по списку
                break

        if best_sk:
            months = v * mult
            # Записываем или обновляем только если найдено большее значение опыта для этого навыка
            if months > req_exp.get(best_sk, 0):
                 req_exp[best_sk] = months
                 proc_spans.add((start, end)) # Отмечаем этот интервал опыта как обработанный

    return req_exp

In [13]:
# Кэширование эмбеддингов навыков
def get_cached_skill_embeddings(skill_db_tuple, sentence_model):
    """
    Создает векторные представления (эмбеддинги) для названий навыков из базы данных.
    Фильтрует некорректные эмбеддинги.
    Возвращает список оригинальных названий навыков, массив их эмбеддингов и карту индексов к нормализованным названиям.
    Принимает словарь навыков в виде кортежа для возможности передачи в основной поток (если необходимо) и модель SentenceTransformer.
    """
    # Лог: Информируем о начале процесса создания эмбеддингов.
    logging.info("Создание эмбеддингов для базы навыков...")
    # Преобразуем входной кортеж пар (ключ, значение) обратно в словарь для удобства работы.
    skill_db = dict(skill_db_tuple)
    # Инициализируем список для хранения оригинальных названий навыков, которые будут кодироваться.
    names = []
    # Инициализируем словарь для сопоставления индекса в списке `names` с нормализованным именем навыка.
    norm_map = {}
    # Инициализируем счетчик индексов.
    idx = 0
    # Итерируем по словарю навыков (интересуют только значения - информация о навыке).
    for _, s_info in skill_db.items():
         # Извлекаем оригинальное название (приоритет русскому, затем английскому) и нормализованное название (аналогично).
         name = s_info['ru'] or s_info['en']
         norm = s_info['norm_ru'] or s_info['norm_en']
         # Если и оригинальное, и нормализованное названия существуют (не пустые строки)...
         if name and norm:
             # ...добавляем оригинальное название в список `names`.
             names.append(name)
             # ...сохраняем соответствие: текущий индекс `idx` -> нормализованное имя `norm`.
             norm_map[idx] = norm
             # ...увеличиваем индекс для следующего навыка.
             idx += 1
    # Если список `names` пуст (не найдено ни одного навыка с названием и нормализацией)...
    if not names:
        # ...возвращаем пустые структуры, так как кодировать нечего.
        return [], None, {}
    # Блок try-except для обработки возможных ошибок во время кодирования.
    try:
        # Получаем эмбеддинги для всех собранных названий навыков с помощью предоставленной модели.
        embs = sentence_model.encode(names, show_progress_bar=False)
        # Получаем ожидаемую размерность эмбеддинга от модели.
        dim = sentence_model.get_sentence_embedding_dimension()
        # Инициализируем списки для хранения валидных эмбеддингов и соответствующих им имен,
        # а также словарь для карты индексов валидных эмбеддингов к нормализованным именам.
        v_embs, f_names, f_map = [], [], {}
        # Инициализируем счетчик индексов для валидных эмбеддингов.
        f_idx = 0
        # Итерируем по полученным эмбеддингам и их исходным индексам `i`.
        for i, emb in enumerate(embs):
            # Проверяем валидность эмбеддинга: должен быть numpy-массивом, иметь правильную размерность,
            # и не содержать некорректных числовых значений (NaN, Inf).
            if isinstance(emb, np.ndarray) and emb.shape == (dim,) and not np.isnan(emb).any() and not np.isinf(emb).any():
                 # Если эмбеддинг валиден:
                 # Добавляем его в список валидных эмбеддингов `v_embs`.
                 v_embs.append(emb)
                 f_names.append(names[i])
                 f_map[f_idx] = norm_map[i]
                 # Увеличиваем счетчик валидных эмбеддингов.
                 f_idx +=1
            else:
                 # Если эмбеддинг невалиден, логируем предупреждение с названием навыка.
                 logging.warning(f"Некорр. эмбеддинг: {names[i]}")
        # Если после фильтрации не осталось ни одного валидного эмбеддинга...
        if not f_names:
            # ...логируем предупреждение и возвращаем пустые структуры.
            logging.warning("Нет валидных эмб. навыков.")
            return [], None, {}
        # Возвращаем:
        # Список оригинальных названий навыков, для которых получены валидные эмбеддинги.
        # Numpy-массив валидных эмбеддингов, преобразованный к типу float32 (требуется для FAISS).
        # Словарь, отображающий новый индекс (0 до N-1) в массиве эмбеддингов на нормализованное имя навыка.
        return f_names, np.array(v_embs).astype('float32'), f_map
    # Перехватываем любые исключения, которые могли произойти во время работы `sentence_model.encode`.
    except Exception as e:
        # Логируем критическую ошибку и возвращаем пустые структуры.
        logging.error(f"Ошибка кодир. эмб. навыков: {e}")
        return [], None, {}

In [14]:
def extract_job_skills_advanced(job_text: str, skill_db: dict, norm_map: dict, nlp_spacy, cached_data) -> list:
    """
    Извлекает список нормализованных навыков из текста вакансии, используя несколько методов:
    1. Распознавание именованных сущностей (NER) с помощью spaCy (если модель доступна).
    2. Поиск семантически близких навыков по эмбеддингам текста вакансии и базы навыков.
    3. Прямой поиск текстовых совпадений названий навыков (и их вариантов) в тексте вакансии.

    """
    # Если текст вакансии пустой, возвращаем пустой список.
    if not job_text: return []
    # Инициализируем множество для хранения найденных нормализованных навыков
    skills_norm = set()
    # Распаковываем кэшированные данные: список оригинальных имен, массив эмбеддингов, карта индексов к нормализованным именам
    names, embs, map_idx_norm = cached_data

    # --- Блок 1: Извлечение с помощью spaCy NER ---
    # Проверяем, была ли успешно загружена модель spaCy.
    if nlp_spacy:
        # Используем try-except для обработки возможных ошибок во время работы spaCy.
        try:
            # Проверяем длину текста: spaCy имеет ограничение на максимальную длину. Обрезаем, если необходимо.
            doc_text = job_text[:nlp_spacy.max_length] if len(job_text) > nlp_spacy.max_length else job_text
            # Логируем предупреждение, если текст был обрезан.
            if len(job_text) > nlp_spacy.max_length:
                logging.warning(f"Текст > {nlp_spacy.max_length} для spaCy, обрезан.")
            # Обрабатываем текст с помощью spaCy для получения объекта Doc с распознанными сущностями.
            doc = nlp_spacy(doc_text)
            # Итерируем по всем найденным именованным сущностям (entities).
            for ent in doc.ents:
                # Проверяем, относится ли метка сущности к интересующим нас типам (технологии, продукты и т.д.).
                if ent.label_ in SPACY_NER_LABELS:
                    # Нормализуем текст найденной сущности (приводим к стандартному виду).
                    n_ent = normalize_skill(ent.text)
                    # Проверяем, есть ли эта нормализованная сущность в нашей карте известных навыков.
                    if n_ent in norm_map:
                        skills_norm.add(n_ent) # Добавляем найденный навык.
                    # Если прямого совпадения нет, проверяем, есть ли синоним этой сущности в карте навыков.
                    elif SKILL_SYNONYM_MAP.get(n_ent) in norm_map:
                        skills_norm.add(SKILL_SYNONYM_MAP.get(n_ent)) # Добавляем основной навык по синониму.
        # Ловим и логируем возможные ошибки при работе spaCy.
        except Exception as e:
            logging.warning(f"Ошибка spaCy NER: {e}")

    # --- Блок 2: Извлечение на основе семантического сходства (KeyBERT-like) ---
    # Используем try-except для обработки ошибок при вычислении эмбеддингов или сходства.
    try:
        # Получаем эмбеддинг (векторное представление) для всего текста вакансии.
        doc_emb = get_embedding(job_text)
        # Проверяем, что эмбеддинг вакансии и кэшированные эмбеддинги навыков существуют и валидны.
        if doc_emb is not None and embs is not None and len(names) > 0:
            # Вычисляем косинусное сходство между эмбеддингом вакансии и эмбеддингами всех навыков из базы.
            sims = cosine_similarity(doc_emb.reshape(1, -1), embs)[0]
            # Находим индексы KEYBERT_TOP_N навыков с наибольшим сходством.
            top_indices = np.argsort(sims)[-KEYBERT_TOP_N:][::-1]
            # Итерируем по индексам самых похожих навыков.
            for idx in top_indices:
                # Проверяем валидность индекса и получаем нормализованное имя навыка по индексу.
                if idx < len(map_idx_norm):
                    n_skill = map_idx_norm.get(idx)
                    # Если навык найден и его сходство с вакансией выше порога (0.35)...
                    if n_skill and sims[idx] > 0.35:
                        skills_norm.add(n_skill) # ...добавляем его в множество найденных навыков.
    # Ловим и логируем возможные ошибки при семантическом поиске.
    except Exception as e:
        logging.warning(f"Ошибка KeyBERT-like: {e}")

    # --- Блок 3: Извлечение на основе прямого текстового совпадения ---
    # Нормализуем текст вакансии для поиска (нижний регистр, удаление лишних пробелов).
    proc_text = normalize_text(job_text)
    # Итерируем по всем известным нормализованным навыкам и их ID из `norm_map`.
    for n_skill, s_id in norm_map.items():
        # Пропускаем, если ID навыка по какой-то причине отсутствует в основной базе `skill_db`.
        if s_id not in skill_db: continue
        # Получаем информацию о навыке (оригинальные русское и английское названия).
        s_info = skill_db[s_id]
        # Создаем множество вариантов написания навыка: нормализованное, русское, английское.
        variants = {n_skill, s_info['ru'], s_info['en']}
        # Проверяем каждый вариант написания.
        for var in variants:
            # Если вариант существует (не пустая строка) и он найден в обработанном тексте вакансии
            # как отдельное слово
            if var and re.search(r'\b' + re.escape(var) + r'\b', proc_text, re.IGNORECASE):
                skills_norm.add(n_skill) # ...добавляем нормализованный навык в результат.
                break # ...и переходим к следующему навыку из `norm_map` (одного совпадения достаточно).

    # Возвращаем отсортированный список уникальных нормализованных навыков, найденных всеми методами.
    return sorted(list(skills_norm))

In [ ]:
def semantic_similarity(t1: str, t2: str, th: float = SEMANTIC_SIMILARITY_THRESHOLD) -> bool:
    """
    Вычисляет семантическое сходство между двумя текстами с использованием эмбеддингов.

    """
    # Возвращаем False, если один из текстов пустой.
    if not t1 or not t2: return False
    # Получаем эмбеддинги (векторные представления) для обоих текстов.
    # Используется кэшированная функция get_embedding.
    e1, e2 = get_embedding(t1), get_embedding(t2);
    # Возвращаем False, если не удалось получить эмбеддинг для одного из текстов.
    if e1 is None or e2 is None: return False
    # Используем try-except для обработки возможных ошибок при вычислении сходства.
    try:
        # Вычисляем косинусное сходство между двумя эмбеддингами.
        # Эмбеддинги преобразуются в numpy массивы и изменяется их форма для функции cosine_similarity.
        sim = cosine_similarity(np.array(e1).reshape(1,-1), np.array(e2).reshape(1,-1))[0][0]
        return sim >= th
    except Exception as e:
        logging.warning(f"Ошибка calc sim '{t1}' vs '{t2}': {e}")
        return False

In [ ]:
# ---- Функции Сравнения ----
def check_synonyms(skill_norm: str, cand_set: set) -> bool:
    """
    Проверяет, присутствует ли нормализованный навык или его синонимы
    в наборе нормализованных навыков кандидата.

    """
    # Ищем основное (каноническое) название для проверяемого навыка в карте синонимов.
    main = SKILL_SYNONYM_MAP.get(skill_norm);
    # Если основное название найдено и оно присутствует у кандидата, возвращаем True.
    if main and main in cand_set: return True
    # Итерируем по всем парам (синоним, основное) в карте синонимов.
    for syn, m in SKILL_SYNONYM_MAP.items():
        # Если основное имя в паре совпадает с проверяемым навыком,
        # и синоним из этой пары присутствует у кандидата, возвращаем True.
        if m == skill_norm and syn in cand_set: return True
    # Если ни навык, ни его синонимы не найдены у кандидата, возвращаем False.
    return False

def calculate_match_details(job_skills: list, spec_skills: list, mandatory_skills: list, orig_map: dict) -> dict:
    """
    Детально сравнивает навыки, требуемые вакансией, с навыками специалиста.
    Учитывает прямые совпадения, синонимы, семантическое сходство и обязательные навыки.

    """
    # Инициализируем словарь результатов значениями по умолчанию.
    res = {'match_percent': 0, 'matched_direct': [], 'matched_synonym': {}, 'matched_semantic': {}, 'missing': []}
    # Если в вакансии не указаны требуемые навыки, возвращаем результат по умолчанию.
    if not job_skills: return res
    # Преобразуем списки навыков во множества для эффективных операций.
    job_set = set(job_skills)       # Навыки вакансии
    cand_set = set(spec_skills)      # Навыки кандидата
    mand_set = set(mandatory_skills) # Обязательные навыки вакансии

    # Инициализируем структуры для хранения результатов совпадений:
    m_dir = set()          # Прямые совпадения
    m_syn = {}             # Совпадения по синонимам
    m_sem = {}             # Семантические совпадения
    missing = set(job_skills)
    missing_mand = False   # Флаг, указывающий на отсутствие обязательного навыка
    cand_avail = cand_set.copy() # Копия навыков кандидата, из которой будем удалять найденные совпадения

    # --- Шаг 1: Проверка обязательных навыков ---
    for mand_skill in mand_set:
        present = False # Флаг наличия текущего обязательного навыка у кандидата
        # 1.1 Проверяем прямое совпадение
        if mand_skill in cand_avail:
            present = True
            # Если найдено: добавляем в прямые совпадения, удаляем из доступных у кандидата и из недостающих.
            m_dir.add(mand_skill)
            cand_avail.remove(mand_skill)
            missing.discard(mand_skill)
        else:
            # 1.2 Проверяем синонимы (если прямого совпадения нет)
            # 1.2.1 Является ли обязательный навык синонимом? Ищем его основную форму.
            main = SKILL_SYNONYM_MAP.get(mand_skill);
            if main and main in cand_avail: # Если основная форма есть у кандидата
                present = True
                m_syn[mand_skill] = main # Записываем совпадение по синониму
                cand_avail.remove(main)    # Удаляем основную форму из доступных у кандидата
                missing.discard(mand_skill)# Удаляем обязательный навык из недостающих
            else:
                # 1.2.2 Является ли обязательный навык основной формой? Ищем его синонимы у кандидата.
                 for syn, m in SKILL_SYNONYM_MAP.items(): # Итерируем по карте синонимов
                     if m == mand_skill and syn in cand_avail: # Если нашли синоним у кандидата
                         present = True
                         m_syn[mand_skill] = syn # Записываем совпадение по синониму
                         cand_avail.remove(syn)   # Удаляем синоним из доступных у кандидата
                         missing.discard(mand_skill) # Удаляем обязательный навык из недостающих
                         break
        # Если после всех проверок обязательный навык не найден у кандидата...
        if not present:
            missing_mand = True # ...устанавливаем флаг отсутствия обязательного навыка
            break # ...и прекращаем проверку обязательных навыков (кандидат не подходит)

    # Если хотя бы один обязательный навык отсутствует, кандидат нерелевантен.
    # Возвращаем результат с 0% совпадения и полным списком требуемых навыков как недостающих.
    if missing_mand:
        res['missing'] = list(job_set)
        return res

    # --- Шаг 2: Проверка остальных (необязательных) навыков ---
    # Получаем множество навыков вакансии, исключая уже проверенные обязательные.
    rem_job = job_set - mand_set
    for skill in rem_job:
        # 2.1 Проверяем прямое совпадение с оставшимися у кандидата навыками
        if skill in cand_avail:
            m_dir.add(skill)
            cand_avail.remove(skill)
            missing.discard(skill)
        else:
            # 2.2 Проверяем синонимы (аналогично проверке обязательных навыков)
            # 2.2.1 Является ли навык синонимом?
            main = SKILL_SYNONYM_MAP.get(skill);
            if main and main in cand_avail:
                m_syn[skill] = main
                cand_avail.remove(main)
                missing.discard(skill)
            else:
                # 2.2.2 Является ли навык основной формой?
                 for syn, m in SKILL_SYNONYM_MAP.items():
                     if m == skill and syn in cand_avail:
                         m_syn[skill] = syn
                         cand_avail.remove(syn)
                         missing.discard(skill)
                         break # Нашли совпадение, выходим из поиска синонимов

    # --- Шаг 3: Поиск семантически близких навыков для оставшихся недостающих ---
    curr_miss = list(missing) # Создаем копию списка недостающих на данный момент навыков для итерации
    for miss_s in curr_miss:
        best_sem = None # Лучший кандидат на семантическое совпадение
        # Получаем оригинальное название недостающего навыка (RU или EN) для сравнения эмбеддингов.
        miss_orig = orig_map.get(miss_s, {});
        miss_txt = miss_orig.get('ru') or miss_orig.get('en') or miss_s # Используем нормализованное, если нет оригинала
        # Итерируем по навыкам, всё еще доступным у кандидата (которые не были засчитаны как прямое или синоним).
        for cand_s_norm in list(cand_avail):
            # Получаем оригинальное название доступного навыка кандидата.
            cand_orig = orig_map.get(cand_s_norm, {});
            cand_txt = cand_orig.get('ru') or cand_orig.get('en') or cand_s_norm
            # Вычисляем семантическое сходство между недостающим навыком и навыком кандидата.
            if semantic_similarity(miss_txt, cand_txt):
                 # Проверяем, не был ли этот навык кандидата уже использован как синоним для другого навыка вакансии.
                 is_syn = any(cand_s == cand_s_norm for _, cand_s in m_syn.items())
                 if not is_syn:
                     best_sem = cand_s_norm
                     break # Нашли первое подходящее семантическое совпадение, переходим к следующему недостающему навыку
        # Если найден семантически близкий навык...
        if best_sem:
            m_sem[miss_s] = best_sem # ...записываем его в результаты семантических совпадений.
            missing.remove(miss_s)   # ...удаляем исходный навык из списка недостающих.
            cand_avail.remove(best_sem) # ...удаляем найденный семантический аналог из доступных у кандидата.

    # --- Шаг 4: Расчет итогового процента совпадения ---
    # Считаем взвешенное количество совпавших навыков: прямые и синонимы дают вес 1.0, семантические - SEMANTIC_MATCH_WEIGHT.
    w_count = (len(m_dir) + len(m_syn)) * 1.0 + len(m_sem) * SEMANTIC_MATCH_WEIGHT
    # Рассчитываем процент совпадения от общего числа требуемых навыков.
    m_perc = (w_count / len(job_skills)) * 100 if job_skills else 0

    # Формируем и возвращаем итоговый словарь с результатами.
    res = {
        'match_percent': round(m_perc, 2), # Округленный процент совпадения
        'matched_direct': sorted(list(m_dir)), # Отсортированный список прямых совпадений
        'matched_synonym': m_syn,         # Словарь совпадений по синонимам
        'matched_semantic': m_sem,       # Словарь семантических совпадений
        'missing': sorted(list(missing))    # Отсортированный список недостающих навыков
    }
    return res

In [17]:
def calculate_experience_match(req_exp: dict, cand_exp: dict) -> tuple[float, list, list]:
    """
    Сравнивает требуемый опыт по навыкам (из вакансии) с опытом кандидата по навыкам.
    Учитывает синонимы навыков при подсчете опыта кандидата.

    """
    # Если словарь требуемого опыта пуст, считаем, что соответствие полное (100%).
    if not req_exp: return 1.0, [], []

    # Инициализация списков для хранения информации о выполненных и невыполненных требованиях.
    met, unmet = [], []
    # Инициализация счетчика выполненных требований и множества для отслеживания уже рассмотренных навыков.
    score, considered = 0, set()
    # Создаем копию словаря опыта кандидата для агрегации опыта по синонимам.
    cand_exp_syn = cand_exp.copy()

    # --- Агрегация опыта кандидата по синонимам ---
    # Итерируем по навыкам кандидата и его опыту в месяцах.
    for cand_n, months in cand_exp.items():
        # Находим основное (каноническое) имя для текущего навыка кандидата.
        main = SKILL_SYNONYM_MAP.get(cand_n)
        # Если найдено основное имя (т.е. текущий навык - синоним)...
        if main:
            # ...добавляем (или обновляем, если уже есть) опыт по основному навыку,
            # выбирая максимальное значение между текущим опытом по основному навыку и опытом по синониму
            cand_exp_syn[main] = max(cand_exp_syn.get(main, 0), months)

    # --- Сравнение требуемого опыта с агрегированным опытом кандидата ---
    # Итерируем по требуемым навыкам и необходимому количеству месяцев опыта.
    for skill_n, req_m in req_exp.items():
        # Если этот требуемый навык уже был рассмотрен (например, как синоним другого), пропускаем.
        if skill_n in considered: continue

        # Получаем опыт кандидата по данному навыку из агрегированного словаря (учитывая синонимы).
        # Если навыка нет, опыт считается равным 0.
        cand_m = cand_exp_syn.get(skill_n, 0)
        # Находим все синонимы для текущего требуемого навыка (skill_n).
        syns = [s for s, m in SKILL_SYNONYM_MAP.items() if m == skill_n]
        # Проверяем также опыт кандидата по каждому из найденных синонимов.
        # Это нужно на случай, если опыт был указан только для синонима в исходных данных кандидата.
        for s in syns:
            cand_m = max(cand_m, cand_exp.get(s, 0)) # Обновляем опыт кандидата максимальным значением

        # Определяем, соответствует ли опыт кандидата (`cand_m`) требуемому (`req_m`).
        is_met = cand_m >= req_m
        # Формируем словарь с детальной информацией о сравнении для этого навыка.
        info = {'skill_norm': skill_n, 'required_months': req_m, 'candidate_months': cand_m, 'met': is_met}

        # Добавляем информацию в соответствующий список (`met` или `unmet`).
        (met if is_met else unmet).append(info);

        # Если требование выполнено, увеличиваем счетчик выполненных требований.
        if is_met: score += 1.0
        # Отмечаем данный требуемый навык как рассмотренный.
        considered.add(skill_n)

    # Рассчитываем итоговую оценку как долю выполненных требований от общего числа требований.
    # Если требований не было (хотя мы это проверили в начале), оценка будет 1.0.
    f_score = score / len(req_exp) if req_exp else 1.0
    # Возвращаем округленную оценку и списки с деталями по выполненным/невыполненным требованиям.
    return round(f_score, 2), met, unmet

In [18]:
# ---- Функции FAISS ----
def prepare_faiss_index(spec_list: list):
    """
    Подготавливает и создает индекс FAISS для быстрого поиска кандидатов по их эмбеддингам.

    """
    # Лог: Начало подготовки индекса FAISS. Засекаем время.
    logging.info("Подготовка индекса FAISS..."); t = time.time()
    # Проверяем, не пуст ли список кандидатов.
    if not spec_list:
        logging.warning("Список кандидатов пуст.") # Лог: Предупреждение, если список пуст.
        return None
    # Используем try-except для перехвата возможных ошибок при создании индекса.
    try:
        # Получаем эмбеддинги для всех текстов кандидатов ('full_text_for_embedding') с помощью модели SentenceTransformer.
        embs = model.encode([s['full_text_for_embedding'] for s in spec_list], show_progress_bar=True, batch_size=32)
        # Преобразуем список эмбеддингов в numpy массив типа float32 (требуется для FAISS).
        embs = np.array(embs).astype('float32')
        embs = np.nan_to_num(embs)
        # Нормализуем векторы по L2-норме
        faiss.normalize_L2(embs)
        # Получаем размерность векторов-эмбеддингов от модели.
        dim = model.get_sentence_embedding_dimension()
        # Создаем плоский индекс FAISS (`IndexFlatIP`), который использует скалярное произведение (Inner Product)
        # для измерения сходства. Подходит для поиска по косинусному сходству с L2-нормализованными векторами.
        idx = faiss.IndexFlatIP(dim)
        # Добавляем нормализованные эмбеддинги кандидатов в индекс.
        idx.add(embs)
        # Лог: Сообщение об успешном создании индекса, его размере (количество векторов) и времени создания.
        logging.info(f"Индекс FAISS ({idx.ntotal}) создан за {time.time()-t:.2f}с.")
        # Возвращаем созданный индекс.
        return idx
    # Перехватываем любые исключения, возникшие при создании индекса.
    except Exception as e:
        # Лог: Сообщение об ошибке при создании индекса FAISS.
        logging.error(f"Ошибка создания FAISS: {e}")
        # Возвращаем None в случае ошибки.
        return None

def search_candidates_faiss(idx, job_txt: str, k: int = 100):
    """
    Ищет k наиболее похожих кандидатов в индексе FAISS на основе текста вакансии.

    """
    # Проверяем, существует ли индекс и не пуст ли текст вакансии.
    if idx is None or not job_txt:
        # Если условия не выполнены, возвращаем пустые списки (преобразованные в массивы далее).
        return [], []
    # Используем try-except для перехвата ошибок во время поиска.
    try:
        # Получаем эмбеддинг для текста вакансии с помощью кэшируемой функции.
        emb = get_embedding(job_txt);
        # Если не удалось получить эмбеддинг для вакансии, возвращаем пустые списки.
        if emb is None: return [],[]
        # Преобразуем эмбеддинг вакансии в numpy массив типа float32,
        # изменяем форму на (1, размерность) для поиска и заменяем NaN/Inf на 0.
        emb = np.array(emb).astype('float32').reshape(1,-1)
        emb = np.nan_to_num(emb)
        # Нормализуем L2-норму эмбеддинга вакансии (так же, как нормализовали векторы в индексе).
        faiss.normalize_L2(emb)
        # Выполняем поиск в индексе FAISS: ищем `k` ближайших векторов к эмбеддингу вакансии.
        # Возвращает оценки сходства (scores) и индексы (indices) найденных кандидатов.
        # Для IndexFlatIP scores - это значения скалярного произведения (близкие к косинусному сходству).
        scores, indices = idx.search(emb, k)
        # FAISS может возвращать -1 в качестве индекса, если найдено меньше k соседей.
        # Создаем маску для фильтрации таких невалидных индексов.
        mask = indices[0] != -1
        # Возвращаем только валидные оценки и индексы, используя созданную маску.
        return scores[0][mask], indices[0][mask]
    # Перехватываем любые исключения во время поиска.
    except Exception as e:
        # Лог: Сообщение об ошибке при поиске в FAISS, указывая начало текста вакансии.
        logging.error(f"Ошибка поиска FAISS '{job_txt[:50]}...': {e}")
        # Возвращаем пустые списки в случае ошибки.
        return [], []

In [20]:
# ---- Основной Цикл ----
if __name__ == "__main__":
    # Лог: Сообщение о старте основного процесса.
    logging.info("--- Запуск основного процесса сопоставления ---")

    # 1. Загрузка данных
    # Вызываем функцию `load_data` для загрузки и предобработки данных из файлов.
    # Явно передаем пути к файлам, определенные в конфигурации.
    job_data, specialists_data, skill_db, normalized_skill_map, original_skill_map_norm = load_data(SKILLS_FILE, JOBS_FILE, SPECIALISTS_FILE)
    # Проверяем, успешно ли загружены основные данные. Если нет - логируем ошибку и завершаем программу.
    if not job_data or not specialists_data or not skill_db:
        logging.error("Крит. ошибка: Не удалось загрузить данные.")
        exit()

    # Кэшируем эмбеддинги навыков
    skill_db_tuple = tuple(sorted(skill_db.items()))
    # Вызываем функцию для получения эмбеддингов навыков. Передаем данные и модель.
    # Результат будет содержать: список имен, массив эмбеддингов, карту индексов к нормализованным именам.
    cached_skill_embeddings_data = get_cached_skill_embeddings(skill_db_tuple, model)
    # Проверяем, удалось ли создать эмбеддинги (второй элемент кортежа - массив эмбеддингов).
    if cached_skill_embeddings_data[1] is None:
        logging.warning("Не удалось кэшировать эмб. навыков.")

    # 2. Подготовка FAISS индекса
    # Создаем индекс FAISS на основе данных специалистов для быстрого поиска по эмбеддингам.
    faiss_index = prepare_faiss_index(specialists_data)
    # Проверяем, был ли успешно создан индекс FAISS. Если нет - логируем ошибку и завершаем программу.
    if faiss_index is None:
        logging.error("Крит. ошибка: Не удалось создать FAISS индекс.")
        exit() # Завершение работы скрипта

    # 3. Сопоставление
    # Инициализируем словарь для хранения результатов сопоставления {job_id: [top_candidates]}.
    results = {}
    # Получаем общее количество вакансий для логирования прогресса.
    total_jobs = len(job_data)
    # Засекаем время начала процесса сопоставления.
    start_matching_time = time.time()

    # Начинаем итерацию по всем вакансиям из `job_data`.
    # `enumerate` используется для получения индекса вакансии (для логирования прогресса).
    for job_idx, (job_id, job_info) in enumerate(job_data.items()):
        # Лог: Начало обработки конкретной вакансии (номер, ID, название).
        logging.info(f"--- Вакансия {job_idx + 1}/{total_jobs}: {job_id} ({job_info['name']}) ---")

        # 3.1: Извлечение требований из вакансии
        # Извлекаем список нормализованных требуемых навыков из текста вакансии.
        req_skills = extract_job_skills_advanced(job_info['full_text'], skill_db, normalized_skill_map, nlp, cached_skill_embeddings_data)
        # Получаем словарь с требованиями к опыту {навык: месяцы}.
        req_exp = job_info['required_experience']
        # Формируем список обязательных навыков (ключи из словаря требований к опыту).
        mand_skills = list(req_exp.keys())
        # Лог: Количество извлеченных навыков. Отладочный лог: Требования к опыту.
        logging.info(f"  Треб. навыки: {len(req_skills)} шт."); logging.debug(f"Треб. опыт: {req_exp}")

        # 3.2: Поиск кандидатов с помощью FAISS (первичный отбор)
        # Используем FAISS для поиска кандидатов, семантически близких к тексту вакансии.
        # Возвращает оценки сходства и индексы кандидатов в `specialists_data`.
        faiss_scores, faiss_indices = search_candidates_faiss(faiss_index, job_info['full_text'])
        # Если FAISS не вернул ни одного кандидата...
        if not len(faiss_indices):
            logging.warning(f"  FAISS не нашел кандидатов.")
            results[job_id] = [] # Записываем пустой список для этой вакансии.
            continue # Переходим к следующей вакансии.
        # Лог: Количество кандидатов, найденных FAISS для детальной проверки.
        logging.info(f"  FAISS вернул {len(faiss_indices)} кандидатов для проверки.")

        # Инициализируем список для хранения обработанных и подходящих кандидатов для текущей вакансии.
        processed_candidates = []
        # Итерируем по индексам кандидатов, найденных FAISS.
        for i in range(len(faiss_indices)):
            # Получаем индекс кандидата и его оценку сходства от FAISS. Преобразуем индекс в int.
            idx, f_score = int(faiss_indices[i]), faiss_scores[i]
            # Дополнительная проверка: индекс должен быть в допустимых пределах списка специалистов.
            if idx < 0 or idx >= len(specialists_data):
                 continue
            # Получаем данные кандидата по его индексу.
            spec = specialists_data[idx]
            logging.debug(f"  Проверка {idx}: {spec['name']} (FAISS: {f_score:.4f})")

            # 3.3: Детальная проверка совпадения по навыкам
            # Вычисляем детали совпадения навыков кандидата с требуемыми навыками вакансии.
            skill_details = calculate_match_details(req_skills, spec['skills_list_norm'], mand_skills, original_skill_map_norm)
            # Если есть обязательные навыки и процент совпадения 0 или меньше (значит, не хватает обязательного навыка)...
            if mand_skills and skill_details['match_percent'] <= 0:
                logging.debug(f"    Отсев (обяз. навыки).")
                continue # Переходим к следующему кандидату.

            # 3.4: Детальная проверка совпадения по опыту
            # Вычисляем соответствие опыта кандидата требуемому опыту.
            exp_score, met_exp, unmet_exp = calculate_experience_match(req_exp, spec['experience_per_skill_months'])
            # Проверяем, есть ли среди невыполненных требований по опыту (`unmet_exp`) обязательные навыки (`mand_skills`).
            if any(unmet['skill_norm'] in mand_skills for unmet in unmet_exp):
                logging.debug(f"    Отсев (обяз. опыт).")
                continue # Переходим к следующему кандидату.

            # 3.5: Расчет итогового балла кандидата
            # Нормализуем процент совпадения навыков к диапазону [0, 1]
            skill_score_norm = skill_details['match_percent'] / 100.0
            # Обеспечиваем, чтобы оценка FAISS была не ниже 0 (на всякий случай)
            safe_f_score = max(0.0, float(f_score))
            # Считаем комбинированную оценку как взвешенную сумму оценок FAISS, навыков и опыта
            comb_score = (W_FAISS * safe_f_score + W_SKILL * skill_score_norm + W_EXPERIENCE * exp_score)

            # --- Форматирование результатов для вывода ---
            # Вспомогательная функция для форматирования словарей совпадений (синонимы, семантика).
            # Заменяет нормализованные имена на оригинальные русские (если есть).
            def fmt_s_dict(md): return sorted([f"{original_skill_map_norm.get(k,{}).get('ru',k)} -> {original_skill_map_norm.get(v,{}).get('ru',v)}" for k,v in md.items()])
            # Вспомогательная функция для форматирования списков навыков (прямые, отсутствующие).
            # Заменяет нормализованные имена на оригинальные русские, убирает дубликаты.
            def fmt_s_list(nl): return sorted(list(set(original_skill_map_norm.get(n,{}).get('ru', n) for n in nl if n)))
            # Вспомогательная функция для форматирования списков требований по опыту (выполненные, невыполненные).
            # Заменяет нормализованные имена на оригинальные русские.
            def fmt_e_list(el): return [{'skill': original_skill_map_norm.get(r['skill_norm'], {}).get('ru', r['skill_norm']), 'req': r['required_months'], 'cand': r['candidate_months']} for r in el]

            # Добавляем информацию об успешно прошедшем проверку кандидате в список `processed_candidates`.
            processed_candidates.append({
                'id': spec['id'],                                        # ID кандидата
                'name': spec['name'],                                    # Имя кандидата
                '_scores': {                                             # Внутренние оценки (для отладки/анализа)
                    'faiss': round(safe_f_score, 4),                     # Оценка FAISS
                    'skill': skill_details['match_percent'],             # Процент совпадения навыков
                    'experience': exp_score                             # Оценка совпадения опыта [0,1]
                },
                'combined_score': round(comb_score, 4),                  # Итоговая комбинированная оценка
                'matching_skills': {                                     # Детали совпадения навыков (отформатированные)
                    'direct': fmt_s_list(skill_details['matched_direct']),
                    'synonym': fmt_s_dict(skill_details['matched_synonym']),
                    'semantic': fmt_s_dict(skill_details['matched_semantic'])
                },
                'missing_skills': fmt_s_list(skill_details['missing']), # Недостающие навыки (отформатированные)
                'experience_compliance': {                               # Детали совпадения опыта (отформатированные)
                    'met': fmt_e_list(met_exp),
                    'unmet': fmt_e_list(unmet_exp)
                }
            })

        # 3.6: Сортировка и сохранение результатов для вакансии
        # Сортируем список обработанных кандидатов по убыванию комбинированной оценки (`combined_score`).
        # Оставляем только топ-5 кандидатов.
        results[job_id] = sorted(processed_candidates, key=lambda x: x['combined_score'], reverse=True)[:5]
        # Лог: Сообщение о количестве найденных релевантных кандидатов для данной вакансии.
        logging.info(f"  -> Найдено {len(results[job_id])} релевантных кандидатов.")

    # Лог: Сообщение о завершении процесса сопоставления и общем времени выполнения.
    logging.info(f"Сопоставление завершено за {time.time() - start_matching_time:.2f} сек.")

    # 4. Пример вывода
    # Выбираем ID вакансии для демонстрации отчета.
    # Приоритет: первая вакансия с непустыми требованиями к опыту.
    # Если таких нет: первая вакансия из результатов.
    # Если результатов нет: None.
    example_job_id = next((j_id for j_id, j_info in job_data.items() if j_info['required_experience']), None) or \
                     (list(results.keys())[0] if results else None)

    # Если удалось выбрать ID вакансии для примера и для нее есть результаты...
    if example_job_id and example_job_id in results:
        logging.info(f"\n--- Пример отчета для вакансии ID: {example_job_id} ---")
        # Получаем данные этой вакансии.
        job_rep = job_data[example_job_id]
        # Повторно извлекаем навыки из текста вакансии (для отчета).
        rep_skills_norm = extract_job_skills_advanced(job_rep['full_text'], skill_db, normalized_skill_map, nlp, cached_skill_embeddings_data)
        # Преобразуем нормализованные навыки в оригинальные русские названия для отчета.
        rep_skills_orig = sorted(list(set(original_skill_map_norm.get(n,{}).get('ru', n) for n in rep_skills_norm if n)))
        # Формируем словарь отчета.
        report = {
            "job_id": example_job_id,
            "job_name": job_rep['name'],
            "required_skills_extracted_orig": rep_skills_orig,
            "required_experience_parsed": [
                {'skill': original_skill_map_norm.get(ns, {}).get('ru', ns), 'months': m}
                for ns, m in sorted(job_rep['required_experience'].items()) # Сортируем по норм. имени для консистентности
            ],
            "top_candidates": results[example_job_id]               # Список топ-5 кандидатов
        }

        # Определяем кастомный JSON-кодировщик для обработки типов данных NumPy (integer, float, ndarray).
        class NpEncoder(json.JSONEncoder):
            def default(self, obj):
                if isinstance(obj, np.integer): return int(obj);
                if isinstance(obj, np.floating): return float(obj);
                if isinstance(obj, np.ndarray): return obj.tolist();
                return super(NpEncoder, self).default(obj)

        # Печатаем отчет в формате JSON
        print(json.dumps(report, indent=2, ensure_ascii=False, cls=NpEncoder))
    else:
        # Лог: Предупреждение, если не удалось сгенерировать пример отчета.
        logging.warning("Результаты пусты или не найден пример.")

    # Лог: Финальное сообщение о завершении работы программы.
    logging.info("--- Работа программы завершена ---")

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

{
  "job_id": "6724df486fa02b99fe3c32b3",
  "job_name": "технический писатель / technical writer",
  "required_skills_extracted_orig": [
    "api",
    "codeigniter",
    "compose",
    "express",
    "git",
    "markdown",
    "rest",
    "rest api",
    "swagger",
    "userstory",
    "winforms",
    "wordstat",
    "ведение документации по проекту",
    "верстка кодом",
    "написание 5-ти руководств по сайту",
    "написание методов для разработчиков",
    "написание проектной документации (тз)",
    "поддержка пользователей",
    "постановка и декомпозиция задач",
    "программирование",
    "работа с git",
    "разработка документации по коду",
    "разработка и доработка функционала",
    "разработка технического задания",
    "разработка технической документации",
    "сдача проекта заказчику",
    "создание и поддержание в актуальном состоянии документации по проектам",
    "техническая документация (техническое задание, функциональные требования, руководство пользователя)"
  